# Multivariante

Hasta ahora, hemos intentado predecir con una variable (`accommodates` y `bedrooms`)

Dos formas de intentar mejorar la precisión:

1. Cambiar el valor de k (cambio de hiperparámetros)
2. Cambiar las variables, incluso incrementando el número de variables, p.ej [`accommodates`, `bedrooms`,  `bathrooms`]


Lo que vamos a hacer en el multivariante es añadir columnas al modelo. Por el momento tienen que ser columnas:

- numéricas
- no nulas (el cálculo de la distancia no nos lo va a permitir)
- no ordinales - tenemos que quitar las variables en las que '2' no represente más que '1', por ejemplo, el id no representa un valor que siendo mayor o menor implique algo; lo mismo con host_id, scrape_id y la latitud y la longitud....

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
valencia = pd.read_csv('https://raw.githubusercontent.com/afoone/caipc-mar-2023/master/data/airbnb.csv')


In [3]:
valencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6711 entries, 0 to 6710
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6711 non-null   int64  
 1   listing_url                                   6711 non-null   object 
 2   scrape_id                                     6711 non-null   int64  
 3   last_scraped                                  6711 non-null   object 
 4   source                                        6711 non-null   object 
 5   name                                          6711 non-null   object 
 6   description                                   6558 non-null   object 
 7   neighborhood_overview                         3759 non-null   object 
 8   picture_url                                   6711 non-null   object 
 9   host_id                                       6711 non-null   i

In [4]:
## limpiar los datos

# corregir el price (tiene $ y no es numérico)
valencia['price'] = valencia['price'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')

valencia = valencia[valencia['price']<700]

In [5]:
# barajar

np.random.seed(3837980)
valencia = valencia.iloc[
    np.random.permutation(len(valencia))
]

In [6]:
valencia.shape

(6493, 75)

In [7]:
valencia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6493 entries, 5738 to 2287
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6493 non-null   int64  
 1   listing_url                                   6493 non-null   object 
 2   scrape_id                                     6493 non-null   int64  
 3   last_scraped                                  6493 non-null   object 
 4   source                                        6493 non-null   object 
 5   name                                          6493 non-null   object 
 6   description                                   6341 non-null   object 
 7   neighborhood_overview                         3567 non-null   object 
 8   picture_url                                   6493 non-null   object 
 9   host_id                                       6493 non-null 

In [10]:
valencia[['bathrooms', 'bedrooms', 'accommodates']]

,bathrooms,bedrooms,accommodates
5738,NaN,1.0,1
1559,NaN,1.0,2
5461,NaN,2.0,4
25,NaN,1.0,3
3559,NaN,2.0,6
...,...,...,...
5882,NaN,NaN,2
2961,NaN,1.0,1
1550,NaN,3.0,6
2312,NaN,1.0,2


In [14]:
valencia['bathrooms'].isnull().sum()

6493

In [15]:
valencia['bathrooms_text']

5738    1.5 shared baths
1559              1 bath
5461              1 bath
25                1 bath
3559              1 bath
              ...       
5882              1 bath
2961      1 private bath
1550              1 bath
2312       1 shared bath
2287              1 bath
Name: bathrooms_text, Length: 6493, dtype: object

In [16]:
valencia['bathrooms_text'].info()

<class 'pandas.core.series.Series'>
Int64Index: 6493 entries, 5738 to 2287
Series name: bathrooms_text
Non-Null Count  Dtype 
--------------  ----- 
6489 non-null   object
dtypes: object(1)
memory usage: 101.5+ KB


In [17]:
valencia['bathrooms_text'].value_counts()

1 bath              2950
2 baths             1087
1 shared bath        957
1.5 baths            483
1 private bath       357
1.5 shared baths     240
2 shared baths       155
2.5 baths             70
3 baths               57
2.5 shared baths      26
3 shared baths        23
4 baths               19
3.5 baths             14
0 shared baths        11
Shared half-bath      10
4 shared baths         8
0 baths                6
5 baths                5
4.5 baths              3
Half-bath              2
5.5 baths              1
4.5 shared baths       1
6.5 baths              1
3.5 shared baths       1
7 baths                1
6 baths                1
Name: bathrooms_text, dtype: int64

In [18]:
def half_bath(x):
    if ('half' in str(x).lower()):
        return '0.5'
    else:
        return x
    

In [19]:
valencia['bathrooms_text']=valencia['bathrooms_text'].apply(
    lambda x: half_bath(x)
) 

In [20]:
valencia['bathrooms_text'].value_counts()

1 bath              2950
2 baths             1087
1 shared bath        957
1.5 baths            483
1 private bath       357
1.5 shared baths     240
2 shared baths       155
2.5 baths             70
3 baths               57
2.5 shared baths      26
3 shared baths        23
4 baths               19
3.5 baths             14
0.5                   12
0 shared baths        11
4 shared baths         8
0 baths                6
5 baths                5
4.5 baths              3
5.5 baths              1
4.5 shared baths       1
6.5 baths              1
3.5 shared baths       1
7 baths                1
6 baths                1
Name: bathrooms_text, dtype: int64

In [23]:
import re

def txt_to_number(x):
    return re.search("[0-9|.]*", str(x)).group(0)

txt_to_number('9.9df')

'9.9'